<a href="https://colab.research.google.com/github/GergelyTuri/sleep/blob/master/tsclustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from os.path import join
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


## Data
dfof: d_awake, d_sleep \\
cosine similarity: ts_dist_awake, ts_dist_sleep



In [3]:
summary_new = pd.read_csv("/gdrive/Shareddrives/Turi_lab/Data/Sleep/2p/Analysis/summary_sleep.csv")
data_path = '/gdrive/Shareddrives/Turi_lab/Data/Sleep/2p/Analysis/data/5HT2afl05b_1/7_12_SD2'
file_name = 'dfof_mc.csv'
dfof_data = pd.read_csv(join(data_path, file_name))
dfof_data.set_index('roi_label', drop=True, inplace=True)

In [4]:
join(data_path, file_name)

'/gdrive/Shareddrives/Turi_lab/Data/Sleep/2p/Analysis/data/5HT2afl05b_1/7_12_SD2/dfof_mc.csv'

In [5]:
def process_dfof_mc(dfof, summary_new):
    n_roi = dfof.shape[0]
    n_time = dfof.shape[1]
    awake = []
    sleep = []
    for j in range(1, 21, 2):  # Adjusted range for Python's 0-indexing
        awake.extend(range(summary_new['start'][j-1], summary_new['end'][j-1]+1))
        sleep.extend(range(summary_new['start'][j], summary_new['end'][j]+1))

    d_awake = dfof.iloc[:, [x-1 for x in awake]]  # Adjust index for Python
    d_sleep = dfof.iloc[:, [x-1 for x in sleep]]
    return {'d_awake': d_awake, 'd_sleep': d_sleep}

In [6]:
result = process_dfof_mc(dfof_data, summary_new)
d_awake = result['d_awake']
d_sleep = result['d_sleep']

In [7]:
from scipy.spatial.distance import cosine
# ts_dist_awake
X = np.array(d_awake)
n = X.shape[0]
dist = np.empty((n, n))
dist.fill(np.nan)

for i in range(n - 1):
    for j in range(i + 1, n):
        dist[i, j] = 1 - cosine(X[i, :], X[j, :])

# Fill the lower triangular part of the matrix to make it symmetric, since cosine distance is symmetric
for i in range(n):
    for j in range(i + 1, n):
        dist[j, i] = dist[i, j]

min_val = np.nanmin(dist)
max_val = np.nanmax(dist)
dist_awake = (dist - min_val) / (max_val - min_val)

# Convert the numpy array to a pandas DataFrame for saving it as CSV
ts_dist_awake = pd.DataFrame(dist_awake)
ts_dist_awake.to_csv('/gdrive/Shareddrives/Turi_lab/Data/Sleep/ts_dist_awake.csv', index=False)

# ts_dist_sleep
X = np.array(d_sleep)
n = X.shape[0]
dist = np.empty((n, n))
dist.fill(np.nan)

for i in range(n - 1):
    for j in range(i + 1, n):
        dist[i, j] = 1 - cosine(X[i, :], X[j, :])

# Fill the lower triangular part of the matrix to make it symmetric, since cosine distance is symmetric
for i in range(n):
    for j in range(i + 1, n):
        dist[j, i] = dist[i, j]

min_val = np.nanmin(dist)
max_val = np.nanmax(dist)
dist_sleep = (dist - min_val) / (max_val - min_val)

# Convert the numpy array to a pandas DataFrame for saving it as CSV
ts_dist_sleep = pd.DataFrame(dist_sleep)
ts_dist_sleep.to_csv('/gdrive/Shareddrives/Turi_lab/Data/Sleep/ts_dist_sleep.csv', index=False)